In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
APP_NAME="acccident_data_process"
from pyspark.sql import functions

In [2]:
import pyspark

In [3]:
import numpy as np

In [4]:
# spark = SparkSession.builder.master("local").appName("Word Count")\
#                 .config("spark.some.config.option", "some-value") \
#                 .getOrCreate()
sc=SparkContext()

In [5]:
sqlContext=SQLContext(sc)

In [6]:
accident_data=sqlContext.read.csv("accident_data_merged.csv",
                                  header=True,encoding="utf-8")

In [7]:
accident_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Unnamed: 0: string (nullable = true)
 |-- accidenttime: string (nullable = true)
 |-- accidentaddr: string (nullable = true)
 |-- driver1fault: string (nullable = true)
 |-- driver1responsibility: string (nullable = true)
 |-- driver2responsibility: string (nullable = true)
 |-- sex1: string (nullable = true)
 |-- platenumber1: string (nullable = true)
 |-- carcolor1: string (nullable = true)
 |-- sex2: string (nullable = true)
 |-- platenumber2: string (nullable = true)
 |-- carcolor2: string (nullable = true)
 |-- jxmc1: string (nullable = true)
 |-- jxmc2: string (nullable = true)
 |-- cclzrq1: string (nullable = true)
 |-- cclzrq2: string (nullable = true)
 |-- clpp1: string (nullable = true)
 |-- clpp2: string (nullable = true)
 |-- driver1license: string (nullable = true)
 |-- driver2license: string (nullable = true)
 |-- brith1: string (nullable = true)
 |-- brith2: string (nullable = true)
 |-- accident_month: string (nullable = true

In [8]:
accident_data=accident_data.drop("Unnamed: 0")
accident_data=accident_data.drop("_c0")

In [9]:
accident_data=accident_data.drop("id_x")
accident_data=accident_data.drop("id_y")
accident_data=accident_data.drop("id")

In [10]:
accident_data=accident_data.replace(["白色","黑色","红色","银色","灰色","黄色","绿色","蓝色","白色","黑 ","北"," 银","BAI","棕色"],
                      ["白","黑","红","银","灰","黄","绿","蓝","白","黑","白","银","白","灰"],["carcolor1","carcolor2"])

In [11]:
colormap2={"白":1,"黑":2,"银":3,"红":4,"蓝":5,"黄":6,"绿":7,"灰":8,"其他":0}
accident_data=accident_data.replace(["其他","白","黑","银","红","蓝","黄","绿","灰"],["0","1","2","3","4","5","6","7","8"],["carcolor1","carcolor2"])

In [12]:
def turn_type(df,cols):
    for col in cols:
        
        df=df.withColumn(col+"Tmp",df[col].cast(pyspark.sql.types.IntegerType())).\
                                    drop(col).withColumnRenamed(col+"Tmp",col)
    return df

In [13]:
# accident_data=turn_type(accident_data,"carcolor2")

In [14]:
weather_map={"多云","阵雨","阴","小雨","晴","中雨","雷阵雨","大雨","暴雨","冻雨"}
accident_data=accident_data.replace(["多云","阵雨","阴","小雨","晴","中雨","雷阵雨","大雨","暴雨","冻雨"],
                                    ["0","1","2","3","4","5","6","7","8","8"],["weather1","weather2"])

In [15]:
# accident_data=turn_type(accident_data,"weather1")

In [16]:
def get_age(age):
    if age:
        age=float(age)
        if (age<30):
            return 0
        elif (age>29 and age<45) :
            return 1
        elif (age>44 and age<60):
            return 2
        else :
            return 3
    else:
        return 3
sparkF=functions.udf(get_age,pyspark.sql.types.IntegerType())

In [17]:
accident_data=accident_data.withColumn("driver1_age_category",sparkF(accident_data.driver1_age))
accident_data=accident_data.withColumn("driver2_age_category",sparkF(accident_data.driver2_age))

In [18]:
def get_year(year):
    if year:
        year=float(year)
        if (year<4):
            return 0
        elif (year>3 and year<11) :
            return 1
        elif (year>10 and year<21):
            return 2
        else :
            return 3
    else:
        return 0
sparkY=functions.udf(get_year,pyspark.sql.types.IntegerType())

In [19]:
accident_data=accident_data.withColumn("driver1_year_category",sparkY(accident_data.driver1_years))
accident_data=accident_data.withColumn("driver2_year_category",sparkY(accident_data.driver2_years))

In [20]:
accident_data.select(["driver1_year_category","driver2_year_category"]).show()

+---------------------+---------------------+
|driver1_year_category|driver2_year_category|
+---------------------+---------------------+
|                    0|                    0|
|                    0|                    0|
|                    0|                    0|
|                    1|                    1|
|                    1|                    1|
|                    1|                    1|
|                    1|                    1|
|                    1|                    1|
|                    0|                    0|
|                    3|                    3|
|                    0|                    0|
|                    2|                    2|
|                    2|                    2|
|                    0|                    0|
|                    0|                    0|
|                    0|                    0|
|                    1|                    1|
|                    2|                    2|
|                    1|           

In [21]:
district_map={"南明区","云岩区","乌当区","花溪区","观山湖区","白云区"}
accident_data=accident_data.replace(["南明区","云岩区","乌当区","花溪区","观山湖区","白云区"],
                                    ["0","1","2","3","4","5"],"district")

In [22]:
accident_data=accident_data.replace(["东北风","南风","东南风","东风"],["1","2","3","4"],["wind1","wind2"])

In [23]:
accident_data=accident_data.replace(["负全部责任","负同等责任"],["1","0"],"driver1responsibility")
accident_data=accident_data.replace(["不负责任","负同等责任"],["1","0"],"driver2responsibility")

In [24]:
# accident_data.show()

In [25]:
accident_data=accident_data.fillna("0","clpp1")
clpp1=accident_data.groupby("clpp1").count()
clpp2=accident_data.groupby("clpp2").count()
clpp1=clpp1.sort("count",ascending=False)
clpp2=clpp2.sort("count",ascending=False)

In [26]:
clpp1=clpp1.select("clpp1")
clpp2=clpp2.select("clpp2")
clpp1_list=clpp1.take(clpp1.count())
clpp2_list=clpp2.take(clpp2.count())

In [27]:
def get_clpp_list(clpp_list,start,end,c):
    clpp=[]
    for i in range(start,end):
        clpp.append(clpp_list[i].asDict()[c])
    return clpp

In [28]:
clpp1_list_divided=list(range(4))
clpp1_list_divided[0]=get_clpp_list(clpp1_list,0,1,"clpp1")
clpp1_list_divided[1]=get_clpp_list(clpp1_list,1,10,"clpp1")
clpp1_list_divided[2]=get_clpp_list(clpp1_list,10,40,"clpp1")
clpp1_list_divided[3]=get_clpp_list(clpp1_list,40,len(clpp1_list),"clpp1")

clpp2_list_divided=list(range(4))
clpp2_list_divided[0]=get_clpp_list(clpp2_list,0,1,"clpp2")
clpp2_list_divided[1]=get_clpp_list(clpp2_list,1,10,"clpp2")
clpp2_list_divided[2]=get_clpp_list(clpp2_list,10,40,"clpp2")
clpp2_list_divided[3]=get_clpp_list(clpp2_list,40,len(clpp2_list),"clpp2")

In [29]:
def get_clpp_num(clpp,clpp_list_divided):
    for i in range(4):
        if clpp in clpp_list_divided[i]:
            return i
        else :continue
# def get_clpp_num2(clpp):
#     for i in range(5):
#         if clpp in clpp2_list_divided[i]:
#             return i
#         else :continue
make_clpp1_function=functions.udf(lambda c :get_clpp_num(c,clpp1_list_divided),pyspark.sql.types.IntegerType())
make_clpp2_function=functions.udf(lambda c :get_clpp_num(c,clpp2_list_divided),pyspark.sql.types.IntegerType())

# sparkC1=functions.udf(get_clpp_num,pyspark.sql.types.IntegerType())
# sparkC2=functions.udf(get_clpp_num,pyspark.sql.types.IntegerType())
accident_data=accident_data.withColumn("clpp1_category",make_clpp1_function(accident_data.clpp1))
accident_data=accident_data.withColumn("clpp2_category",make_clpp2_function(accident_data.clpp1))

In [30]:
accident_data.select(["clpp1_category","clpp2_category"]).show()

+--------------+--------------+
|clpp1_category|clpp2_category|
+--------------+--------------+
|             2|             2|
|             1|             1|
|             1|             1|
|             2|             2|
|             3|             3|
|             3|             3|
|             1|             1|
|             2|             2|
|             2|             2|
|             1|             1|
|             1|             1|
|             1|             2|
|             2|             2|
|             0|             0|
|             3|             3|
|             1|             1|
|             2|             2|
|             1|             1|
|             0|             0|
|             3|             3|
+--------------+--------------+
only showing top 20 rows



In [31]:
accident_data.select(["clpp1_category","clpp2_category"]).show()

+--------------+--------------+
|clpp1_category|clpp2_category|
+--------------+--------------+
|             2|             2|
|             1|             1|
|             1|             1|
|             2|             2|
|             3|             3|
|             3|             3|
|             1|             1|
|             2|             2|
|             2|             2|
|             1|             1|
|             1|             1|
|             1|             2|
|             2|             2|
|             0|             0|
|             3|             3|
|             1|             1|
|             2|             2|
|             1|             1|
|             0|             0|
|             3|             3|
+--------------+--------------+
only showing top 20 rows



In [32]:
jxmc1=accident_data.groupby("jxmc1").count()
jxmc2=accident_data.groupby("jxmc2").count()

In [33]:
jxmc1=jxmc1.sort("count",ascending=False)
jxmc2=jxmc2.sort("count",ascending=False)

In [34]:
jxmc1=jxmc1.select("jxmc1")
jxmc2=jxmc2.select("jxmc2")
jxmc1_list=jxmc1.take(jxmc1.count())
jxmc2_list=jxmc2.take(jxmc2.count())

In [35]:
def get_jxmc_list(jxmc_list,start,end,c):
    jxmc=[]
    for i in range(start,end):
        jxmc.append(jxmc_list[i].asDict()[c])
    return jxmc

In [36]:
jxmc1_list_divided=list(range(5))
jxmc2_list_divided=list(range(5))
jxmc1_list_divided[0]=get_jxmc_list(jxmc1_list,0,1,"jxmc1")
jxmc1_list_divided[1]=get_jxmc_list(jxmc1_list,1,2,"jxmc1")
jxmc1_list_divided[2]=get_jxmc_list(jxmc1_list,2,10,"jxmc1")
jxmc1_list_divided[3]=get_jxmc_list(jxmc1_list,10,40,"jxmc1")
jxmc1_list_divided[4]=get_jxmc_list(jxmc1_list,40,len(jxmc1_list),"jxmc1")
jxmc2_list_divided[0]=get_jxmc_list(jxmc2_list,0,1,"jxmc2")
jxmc2_list_divided[1]=get_jxmc_list(jxmc2_list,1,2,"jxmc2")
jxmc2_list_divided[2]=get_jxmc_list(jxmc2_list,2,10,"jxmc2")
jxmc2_list_divided[3]=get_jxmc_list(jxmc2_list,10,40,"jxmc2")
jxmc2_list_divided[4]=get_jxmc_list(jxmc2_list,40,len(jxmc2_list),"jxmc2")

In [37]:
def get_jxmc_num1(jxmc):
    for i in range(5):
        if jxmc in jxmc1_list_divided[i]:
            return i
        else :continue
def get_jxmc_num2(jxmc):
    for i in range(5):
        if jxmc in jxmc2_list_divided[i]:
            return i
        else :continue
sparkJ1=functions.udf(get_jxmc_num1,pyspark.sql.types.IntegerType())
sparkJ2=functions.udf(get_jxmc_num1,pyspark.sql.types.IntegerType())
accident_data=accident_data.withColumn("jxmc1_category",sparkJ1(accident_data.jxmc1))
accident_data=accident_data.withColumn("jxmc2_category",sparkJ2(accident_data.jxmc1))

In [38]:
accident_data.select(["jxmc1_category","jxmc2_category"]).show()

+--------------+--------------+
|jxmc1_category|jxmc2_category|
+--------------+--------------+
|             1|             1|
|             4|             4|
|             3|             3|
|             2|             2|
|             0|             0|
|             3|             3|
|             0|             0|
|             1|             1|
|             2|             2|
|             3|             3|
|             2|             2|
|             1|             1|
|             0|             0|
|             2|             2|
|             0|             0|
|             1|             1|
|             1|             1|
|             0|             0|
|             0|             0|
|             2|             2|
+--------------+--------------+
only showing top 20 rows



In [39]:
accident_data.select("driver1_age_category").show()

+--------------------+
|driver1_age_category|
+--------------------+
|                   1|
|                   1|
|                   2|
|                   2|
|                   1|
|                   1|
|                   1|
|                   3|
|                   1|
|                   1|
|                   2|
|                   3|
|                   1|
|                   1|
|                   1|
|                   1|
|                   1|
|                   1|
|                   1|
|                   2|
+--------------------+
only showing top 20 rows



In [40]:
# a=accident_data.select(pyspark.sql.functions.mean(accident_data.driver1_years))

In [41]:
# s=pyspark.sql.functions.mean(accident_data.driver1_years)

In [42]:
accident_data=accident_data.withColumn("temperature",(accident_data.temperature_min+accident_data.temperature_max)/2)

In [43]:
accident_data.select("temperature").show()

+-----------+
|temperature|
+-----------+
|        9.0|
|        7.0|
|        7.0|
|        4.5|
|        4.0|
|        3.0|
|        5.0|
|        5.0|
|        5.0|
|        5.0|
|        5.0|
|        5.0|
|        5.5|
|        5.5|
|        5.5|
|        5.5|
|        5.5|
|        7.0|
|        6.5|
|        6.5|
+-----------+
only showing top 20 rows



In [44]:
accident_data.printSchema()

root
 |-- accidenttime: string (nullable = true)
 |-- accidentaddr: string (nullable = true)
 |-- driver1fault: string (nullable = true)
 |-- driver1responsibility: string (nullable = true)
 |-- driver2responsibility: string (nullable = true)
 |-- sex1: string (nullable = true)
 |-- platenumber1: string (nullable = true)
 |-- carcolor1: string (nullable = true)
 |-- sex2: string (nullable = true)
 |-- platenumber2: string (nullable = true)
 |-- carcolor2: string (nullable = true)
 |-- jxmc1: string (nullable = true)
 |-- jxmc2: string (nullable = true)
 |-- cclzrq1: string (nullable = true)
 |-- cclzrq2: string (nullable = true)
 |-- clpp1: string (nullable = false)
 |-- clpp2: string (nullable = true)
 |-- driver1license: string (nullable = true)
 |-- driver2license: string (nullable = true)
 |-- brith1: string (nullable = true)
 |-- brith2: string (nullable = true)
 |-- accident_month: string (nullable = true)
 |-- accident_quarter: string (nullable = true)
 |-- accident_weekday: str

In [45]:
cols=["driver1fault","sex1","carcolor1","clpp1","jxmc1",
      "sex2","carcolor2","clpp2","jxmc2",
      "accident_month","accident_quarter","accident_weekday","accident_day","accident_hour","accident_minute",
      "is_province1","is_city1","is_driver1_city","is_driver1_province",
      "is_province2","is_city2","is_driver2_city","is_driver2_province",
      "weather1","weather2","wind1","wind2","district","lng","lat",
       "fine_x", "score_x", "maxtime_x", "xfcount_x", "wfxw_x","fine_y", "score_y", "maxtime_y", "xfcount_y", "wfxw_y"]
accident_data=turn_type(accident_data,cols)

In [54]:
accident_data=accident_data.fillna(0,["fine_x", "score_x","xfcount_x", "wfxw_x",
                                      "fine_y", "score_y","xfcount_y", "wfxw_y","district","carcolor1"])

In [55]:
accident_data.printSchema()

root
 |-- accidenttime: string (nullable = true)
 |-- accidentaddr: string (nullable = true)
 |-- driver1responsibility: string (nullable = true)
 |-- driver2responsibility: string (nullable = true)
 |-- platenumber1: string (nullable = true)
 |-- platenumber2: string (nullable = true)
 |-- cclzrq1: string (nullable = true)
 |-- cclzrq2: string (nullable = true)
 |-- driver1license: string (nullable = true)
 |-- driver2license: string (nullable = true)
 |-- brith1: string (nullable = true)
 |-- brith2: string (nullable = true)
 |-- is_private: string (nullable = true)
 |-- end_number1: string (nullable = true)
 |-- end_number2: string (nullable = true)
 |-- is_private1: string (nullable = true)
 |-- is_private2: string (nullable = true)
 |-- driver1_days: string (nullable = true)
 |-- driver2_days: string (nullable = true)
 |-- driver1_years: string (nullable = true)
 |-- driver2_years: string (nullable = true)
 |-- driver1_age: string (nullable = true)
 |-- driver2_age: string (nullab

In [56]:
driver1=accident_data.select(["driver1fault","sex1","carcolor1","clpp1_category",
             "jxmc1_category","accident_month","accident_weekday",
             "accident_day","accident_hour","accident_minute","is_province1",
             "is_city1","is_driver1_city","is_driver1_province",
             "temperature",
             "weather1","weather2","wind1","wind2","district","lng","lat",
             "driver1_age_category","driver1_year_category",
              "fine_x", "score_x", "maxtime_x", "xfcount_x", "wfxw_x"
             ])

In [57]:
driver1.printSchema()

root
 |-- driver1fault: integer (nullable = true)
 |-- sex1: integer (nullable = true)
 |-- carcolor1: integer (nullable = true)
 |-- clpp1_category: integer (nullable = true)
 |-- jxmc1_category: integer (nullable = true)
 |-- accident_month: integer (nullable = true)
 |-- accident_weekday: integer (nullable = true)
 |-- accident_day: integer (nullable = true)
 |-- accident_hour: integer (nullable = true)
 |-- accident_minute: integer (nullable = true)
 |-- is_province1: integer (nullable = true)
 |-- is_city1: integer (nullable = true)
 |-- is_driver1_city: integer (nullable = true)
 |-- is_driver1_province: integer (nullable = true)
 |-- temperature: double (nullable = true)
 |-- weather1: integer (nullable = true)
 |-- weather2: integer (nullable = true)
 |-- wind1: integer (nullable = true)
 |-- wind2: integer (nullable = true)
 |-- district: integer (nullable = true)
 |-- lng: integer (nullable = true)
 |-- lat: integer (nullable = true)
 |-- driver1_age_category: integer (nullab

In [60]:
from pyspark.ml.classification import RandomForestClassifier

In [61]:
help(RandomForestClassifier)

Help on class RandomForestClassifier in module pyspark.ml.classification:

class RandomForestClassifier(pyspark.ml.wrapper.JavaEstimator, pyspark.ml.param.shared.HasFeaturesCol, pyspark.ml.param.shared.HasLabelCol, pyspark.ml.param.shared.HasPredictionCol, pyspark.ml.param.shared.HasSeed, pyspark.ml.param.shared.HasRawPredictionCol, pyspark.ml.param.shared.HasProbabilityCol, pyspark.ml.regression.RandomForestParams, TreeClassifierParams, pyspark.ml.param.shared.HasCheckpointInterval, pyspark.ml.util.JavaMLWritable, pyspark.ml.util.JavaMLReadable)
 |  `Random Forest <http://en.wikipedia.org/wiki/Random_forest>`_
 |  learning algorithm for classification.
 |  It supports both binary and multiclass labels, as well as both continuous and categorical
 |  features.
 |  
 |  >>> import numpy
 |  >>> from numpy import allclose
 |  >>> from pyspark.ml.linalg import Vectors
 |  >>> from pyspark.ml.feature import StringIndexer
 |  >>> df = spark.createDataFrame([
 |  ...     (1.0, Vectors.dense(1